## Imports

In [1]:
from ipynb.fs.full.Jacobian import *

import numpy as np
from numpy import linalg as LA
import sympy as sp

import matplotlib.pyplot as plt
import math

## Global Variables

In [234]:
#Time variables
interval = 10
delta = 0.01

#Numbers represent how far we want to be precised.
Epsilon = 10**-2
Nnewton = 10

## Transitionals functions
### Solver with one equation

In [3]:
"""
    Create the function f wich correspond to this equality : du/dt = expr(u, v). 
    Here du/dt -> un - un-1 / delta
    
    @return: the new function ready to be evaluated.
    
    @var: the symbol we want to evalute (u or v)
    @term_1: the previous term
    @expr: the function tied.
"""

def create_f(var, term_1, expr):
    return (var - term_1)/delta - expr

In [4]:
"""
    Create a list like that : [[t0, u0], [t1, u1], ..., [tn, un]]

    @return: [[t0, u0], [t1, u1], ..., [tn, un]]
    
    @time: [t0, t1, ...]
    @function: [u0, u1, ...]
"""

def result_one_eq(time, function):
    final_points = []
    for i in range(len(function)):
        final_points.append([time[i], function[i]])
        
    return final_points

### Solver with 2 equations

In [14]:
"""
    Same as create_f. The only difference is that this time we 
    keep it in a vector cause we have two different functions.
    
    @return: a vector containing the 2 new functions ready to be evaluated.
    
    @xn_1_global: previous term
    @f1: first function
    @f2 second function
"""

def create_2_f(xn_1_global, f1, f2):
    return np.array([create_f(u, xn_1_global[0], f1), create_f(v, xn_1_global[1], f2)])

In [53]:
"""
    Create the 2x2 matrix wich correspond to the derivate of F(xk) 
    when xk is vector like xk = (f1, f2).
    This matrix = [df1/du , df2/du]
                  [df1/dv , df2/dv]
    
    @return: new matrix created.
    
    @xk_1: previous term
    @f1: first function
    @f2: second function
"""

def f_prime_2_eqs(xk_1, f1, f2):
    res = np.array([[0., 0.], [0., 0.]])
    symbols = [u, v]
    tab_f = [f1, f2]
    
    for i in range(2):
        for j in range(2):
            res[i][j] = Jacobian_square_mat(symbols[i], tab_f[j]).subs(symbols[i], xk_1[j])
    
    return res

In [232]:
"""
    Create the term F(xk_1) wich is a vector of size 2 containing the evaluation
    respectivly of f1 and f2.
    
    @return: the term F(xk_1)
    
    @xk_1 previous term
    @vec_f: list containing f1 and f2
"""

def create_fxk_1(xk_1, vec_f):
    res = np.array([0., 0.])
    for i in range(2):
        
        res[i] = vec_f[i].evalf(subs={u: xk_1[0], v: xk_1[1]})
    
    return res

In [8]:
"""
    Create a list containings the lists of points to build the u function
    and the v function.
    
    @return: [ [ [t0, u0], [t1, u1], ..., [tn, un] ] , [ [t0, v0], [t1, v1], ..., [tn, vn] ] ]
    
    @time: list containing t0, t1, t2, ...
    @fu: list containing u0, u1, u2, ...
    @fv: list containing v0, v1, v2, ...
"""

def result_2_eq(time, fu, fv):
    #Creation of our final list containing tuples like that: (time, function)
    final_points_u = []
    final_points_v = []
    
    for i in range(len(fu)):
        final_points_u.append([time[i], fu[i]])
        final_points_v.append([time[i], fv[i]])
        
    return [final_points_u, final_points_v]

In [196]:
"""
    Create a list containings the lists of points to build the u function
    and the v function + x and y variables
    
    @return: [ [ [t0, u0], [t1, u1], ..., [tn, un] ] , [ [t0, v0], [t1, v1], ..., [tn, vn] ] ] 
    + x and y points's list
    
    @time: list containing t0, t1, t2, ...
    @fu: list containing u0, u1, u2, ...
    @fv: list containing v0, v1, v2, ...
    @x: containing all the x calculated
    @y: containing all the y calculated
"""

def result_4_eq(time, fu, fv, x, y):
    #Creation of our final list containing tuples like that: (time, function)
    final_points_u = []
    final_points_v = []
    final_points_x = []
    final_points_y = []
    
    for i in range(len(fu)):
        final_points_u.append([time[i], fu[i]])
        final_points_v.append([time[i], fv[i]])
        final_points_x.append([time[i], x[i]])
        final_points_y.append([time[i], y[i]])
        
    return [final_points_u, final_points_v, final_points_x, final_points_y]

## Solver for one differencial equation

In [9]:
def Solver_one_equation(expr, u0):
    t0 = 0
    i = 1
    f_uk = 1
    uk_1 = u0
    nb = 0
    
    #Each list is an axis.
    #Time = [t0, t1, t2, ...., tn]
    #Function = [u0, u1, u2, ...., un]
    function = [u0]
    time = [t0]
    
    first = True
    
    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        #Newton's method
        while(f_uk > Epsilon or nb < Nnewton):
            
            f_uk_1 = 0
            f_uk_1_prime = 0
            
            # First step, we change our equation to f(x) = 0
            f = create_f(u, uk_1, expr)
            
            # We recup the term f(uk-1)
            if(first):
                f_uk_1 = f.subs(u, uk_1)
                first = False
            
            else:
                f_uk_1 = f.subs(u, function[i-1])
            
            #We recup the term f'(uk_1)
            f_uk_1_prime = Jacobian_one(f).subs(u, uk_1)

            #Formula : uk = uk_1 - (f(uk_1) / f'(uk_1))
            uk = uk_1 - (f_uk_1 / f_uk_1_prime)

            #Check the approximation. If f_uk < Epsilon = good approximation
            f_uk = f.subs(u, uk)
            
            uk_1 = uk
            if(f_uk < Epsilon):
                break
        
        #Update our 2 lists and indexes.
        time.append(t0)
        function.append(uk)
        f_uk = 1
        i += 1
        nb += 1

    return result_one_eq(time, function)

## Solver for 2 differentials equations

In [225]:
def Solver_two_equations(u0, v0, f1, f2):
    t0 = 0
    f_xk = 1
    uk_1 = u0
    vk_1 = v0
    xk_1 = np.array([u0, v0])
    xn_1_global = np.array([u0, v0])
    
    #Each list is an axis.
    #Time = [t0, t1, t2, ...., tn]
    #Functions = [u0, u1, u2, ...., un]
    fu = [u0]
    fv = [v0]
    time = [t0]
    
    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        
        #Newton's method
        while(nb < Nnewton):
            nb+=1
            
            # First step, we change our equation to F(x) = 0
            vec_f = create_2_f(xn_1_global, f1, f2)
            
            #Creating F(xk-1)
            F_xk_1 = create_fxk_1(xk_1, vec_f)
            
            #Creating F'(xk-1)
            F_xk_1_prime = f_prime_2_eqs(xk_1, vec_f[0], vec_f[1])
        
            
            #Formula : xk = xk_1 - tr(F'(xk_1)) * F(xk_1)
            xk = xk_1 - np.dot(LA.inv(F_xk_1_prime), F_xk_1)
          
            #2 types of check the approximation.
            f_xk =[0, 0]
            f_xk[0] = float(vec_f[0].evalf(subs={u: xk[0], v: xk[1]}))
            f_xk[1] = float(vec_f[1].evalf(subs={u: xk[0], v: xk[1]})) 
            norm_f_xk = LA.norm(f_xk)
            norm_xk = LA.norm(xk - xn_1_global)
            if(norm_xk < Epsilon or norm_f_xk < Epsilon):
                break
            
            xk_1 = xk
        
        #Update our 3 lists and indexes.
        time.append(t0)
        fu.append(xk[0])
        fv.append(xk[1])
        
        xn_1_global = np.array([xk[0],xk[1]])
        f_xk = 1        
      
    return result_2_eq(time, fu, fv)

## Solver for 4 differentials equations

In [233]:
def Solver_4_equations(u0, v0, f1, f2):
    t0 = 0
    f_xk = 1
    uk_1 = u0
    vk_1 = v0
    xk_1 = np.array([u0, v0])
    xn_1_global = np.array([u0, v0])
    
    #Each list is an axis.
    #Time = [t0, t1, t2, ...., tn]
    #Functions = [u0, u1, u2, ...., un]
    fu = [u0]
    fv = [v0]
    time = [t0]
    
    #Suit of x and y variables
    y0 = u0*v0
    x0 = u0 + v0 - y0
    x = [x0]
    y = [y0]
    
    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        
        #Newton's method
        while(nb < Nnewton):
            nb+=1
            
            # First step, we change our equation to F(x) = 0
            vec_f = create_2_f(xn_1_global, f1, f2)
            
            #Creating F(xk-1)
            F_xk_1 = create_fxk_1(xk_1, vec_f)
            
            #Creating F'(xk-1)
            F_xk_1_prime = f_prime_2_eqs(xk_1, vec_f[0], vec_f[1])
        
            
            #Formula : xk = xk_1 - tr(F'(xk_1)) * F(xk_1)
            xk = xk_1 - np.dot(LA.inv(F_xk_1_prime), F_xk_1)
          
            #Calcul of f(uk, vk, xk, yk) for the approximation
            f_xk =[0, 0, 0, 0]
            next_y = xk[0]*xk[1]
            next_x = xk[0] + xk[1] - next_y
            f_xk[0] = float(vec_f[0].evalf(subs={u: xk[0], v: xk[1]}))
            f_xk[1] = float(vec_f[1].evalf(subs={u: xk[0], v: xk[1]}))
            f_xk[2] = float(next_x)
            f_xk[3] = float(next_y)
            
            #2 types of check the approximation.
            norm_f_xk = LA.norm(f_xk)
            norm_xk = LA.norm(xk - xn_1_global)
            if(norm_xk < Epsilon):
                break
            
            xk_1 = xk
        
        #Update our 2 functions + time
        time.append(t0)
        fu.append(xk[0])
        fv.append(xk[1])
        
        #Update x and y variables
        x.append(next_x)
        y.append(next_y)
            
        #Update indexes
        xn_1_global = np.array([xk[0],xk[1]])
        f_xk = 1        
      
    return result_4_eq(time, fu, fv, x, y)